# Continual Learning Pipeline implementation on MNIST Dataset

**Continual learning** (also known as **lifelong learning**) refers to the ability of a machine learning model or system to continuously learn and adapt to new information or tasks over time, without forgetting previously learned knowledge. Unlike traditional machine learning, which typically trains on a fixed dataset and then remains static, continual learning allows models to evolve and improve as they are exposed to new data.

Key characteristics of continual learning include:

1. **Learning from New Data**: The model is capable of incorporating new information or tasks while maintaining performance on earlier tasks.
  
2. **Avoiding Catastrophic Forgetting**: A major challenge in continual learning is ensuring that the model doesn't forget previously learned tasks when learning new ones.

3. **Adaptation to Changing Environments**: The model can adjust to changes in data distribution or new tasks that weren't initially part of its training.

4. **Efficiency**: The system should be able to learn and update efficiently, often with limited computational resources and without requiring retraining from scratch.

In practice, continual learning is important in areas such as robotics, autonomous systems, and real-time data analysis, where new experiences or information continually emerge.

# Implementation using Avalanche

**Avalanche** is a framework designed to support **continual learning** in machine learning. It provides tools and libraries for building, testing, and evaluating continual learning models, helping researchers and practitioners develop algorithms that can learn over time without forgetting previously acquired knowledge.

### Key Features of Avalanche:
1. **Task-Incremental Learning**: It supports various types of continual learning paradigms, such as task-incremental learning, where the model encounters new tasks over time and needs to retain knowledge from previous tasks.
  
2. **Support for Different Scenarios**: Avalanche provides flexibility in handling different scenarios in continual learning, including:
   - **Class-incremental learning**: Where classes are introduced one by one.
   - **Domain-incremental learning**: Where the domain or distribution changes over time, such as in changing environments or evolving data distributions.

3. **Benchmarking**: The framework offers tools to evaluate models on several well-established continual learning benchmarks, allowing researchers to test the effectiveness of their models and compare them against others in the field.

4. **Memory Management**: Since continual learning often involves a challenge of avoiding **catastrophic forgetting**, Avalanche provides tools to implement memory mechanisms, such as replay buffers, to help retain useful information from past tasks.

5. **Flexible Integration**: Avalanche is compatible with popular deep learning libraries like PyTorch, making it easy to integrate into existing workflows.

6. **Metrics and Evaluation**: The framework offers a range of evaluation metrics, such as accuracy, forgetting, and others that are specifically useful in continual learning scenarios.

### Why Avalanche is Important:
Continual learning poses several challenges, notably **catastrophic forgetting**, where models forget previously learned tasks or knowledge when exposed to new tasks. Avalanche aids in developing and testing solutions to this problem by providing a structured environment to experiment with strategies such as:
- **Replay methods**, where previously encountered data or tasks are replayed to avoid forgetting.
- **Regularization techniques**, which help preserve older knowledge while allowing the model to adapt to new tasks.

By using Avalanche, researchers can advance our understanding of how models can learn continuously and effectively over time, making it a valuable tool in the field of **lifelong learning**.

# Installation

In [ ]:
! pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-kbpwvza1
  Running command git clone --filter=blob:none --quiet https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-kbpwvza1
  Resolved https://github.com/ContinualAI/avalanche.git to commit 625e46d9203878ed51457f6d7cd8d4e9fb05d093
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.2/134.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.2/585.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.6/558.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

# Implementation

Importing Libraries

In [ ]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive


Creating Benchmark

In [ ]:
# here it is creating benchmark using 5 tasks as [0,1],[2,3],[4,5],[6,7],[8,9]

scenario = SplitMNIST(n_experiences=5)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 35.8MB/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.36MB/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 10.9MB/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.06MB/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



Create The Model

In [ ]:
model = SimpleMLP(num_classes=scenario.n_classes)


Define Evaluation Plugin and Loggers

In [ ]:
# Loggers to record and visualize metrics
tb_logger = TensorboardLogger()
# for storing logs in text file
text_logger = TextLogger(open('log.txt', 'a'))
# printing Logs
interactive_logger = InteractiveLogger()

# Evaluation plugin to manage and log metrics
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)


# Define the Naive Strategy
**Naive**: This is a basic strategy for training a model on each task. In continual learning, the Naive strategy doesn't use any techniques like replay or regularization to prevent forgetting. It just trains the model on the current task.

**SGD**: Stochastic Gradient Descent optimizer with a learning rate of 0.001 and momentum of 0.9.

CrossEntropyLoss: Loss function used for classification tasks.

train_mb_size=500: The mini-batch size during training (500 samples).

train_epochs=1: The number of epochs (1 epoch per experience).

eval_mb_size=100: The mini-batch size during evaluation.

evaluator=eval_plugin: The evaluation plugin to track and log metrics during training.

In [ ]:
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)


/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


# Training Loop

In [ ]:
# Start training loop
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # Train on the current experience
    res = cl_strategy.train(experience)
    print('Training completed')

    # Evaluate on the entire test set
    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(scenario.test_stream))


Starting experiment...
Start of experience:  0
Current Classes:  [0, 1]
-- >> Start of training phase << --
100%|██████████| 26/26 [00:03<00:00,  8.36it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55612.8779
	DiskUsage_MB/train_phase/train_stream/Task000 = 55612.8779
	Loss_Epoch/train_phase/train_stream/Task000 = 0.9452
	Loss_MB/train_phase/train_stream/Task000 = 0.1016
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0113
	Time_Epoch/train_phase/train_stream/Task000 = 3.1028
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8131
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 22/22 [00:00<00:00, 31.17it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 86.5873
	DiskUsage_Exp/eval


# Explanation
As we have seen above we used simple Naive approach and the model accuracy decreases on previoulsy learned classes so we are going to use some CL approaches to stop Forgetting

# **GroupBalancedDataLoader**

GroupBalancedDataLoader takes a sequence of datasets and iterates over them by providing balanced mini-batches, where the number of samples is split equally among groups. Internally, it instantiate a DataLoader for each separate group.

In [ ]:
from avalanche.benchmarks import SplitMNIST
from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader
benchmark = SplitMNIST(5, return_task_id=True)

dl = GroupBalancedDataLoader([exp.dataset for exp in benchmark.train_stream], batch_size=5)
for x, y, t in dl:
    print(t.tolist())
    break

[0, 1, 2, 3, 4]


# **ReservoirSamplingBuffer**

Memory buffers store data up to a maximum capacity, and they implement policies to select which data to store and which the to remove when the buffer is full.

In [ ]:
from avalanche.training.storage_policy import ReservoirSamplingBuffer
from types import SimpleNamespace

benchmark = SplitMNIST(5, return_task_id=False)
storage_p = ReservoirSamplingBuffer(max_size=30)

print(f"Max buffer size: {storage_p.max_size}, current size: {len(storage_p.buffer)}")

Max buffer size: 30, current size: 0


 We use a SimpleNamespace because we want to use the buffer standalone, without instantiating an Avalanche strategy. Reservoir sampling requires only the experience from the strategy's state.

In [ ]:
for i in range(5):
    strategy_state = SimpleNamespace(experience=benchmark.train_stream[i])
    storage_p.update(strategy_state)
    print(f"Max buffer size: {storage_p.max_size}, current size: {len(storage_p.buffer)}")
    print(f"class targets: {storage_p.buffer.targets.uniques}\n")

Max buffer size: 30, current size: 30
class targets: {4, 7}

Max buffer size: 30, current size: 30
class targets: {1, 4, 6, 7}

Max buffer size: 30, current size: 30
class targets: {1, 3, 4, 6, 7, 9}

Max buffer size: 30, current size: 30
class targets: {1, 3, 4, 5, 6, 7, 8, 9}

Max buffer size: 30, current size: 30
class targets: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}



<ipython-input-11-b534ddf892b6>:3: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-11-b534ddf892b6>:3: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-11-b534ddf892b6>:3: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-11-b534ddf892b6>:3: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-11-b534ddf892b6>:3: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)


# **ParametricBuffer**

ParametricBuffer is a buffer split into several groups according to the groupby parameters (None, 'class', 'task', 'experience'), and according to an optional ExemplarsSelectionStrategy (random selection is the default choice).

In [ ]:
from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy
storage_p = ParametricBuffer(
    max_size=30,
    groupby='class',
    selection_strategy=RandomExemplarsSelectionStrategy()
)

print(f"Max buffer size: {storage_p.max_size}, current size: {len(storage_p.buffer)}")
for i in range(5):
    strategy_state = SimpleNamespace(experience=benchmark.train_stream[i])
    storage_p.update(strategy_state)
    print(f"Max buffer size: {storage_p.max_size}, current size: {len(storage_p.buffer)}")
    print(f"class targets: {storage_p.buffer.targets.uniques}\n")

Max buffer size: 30, current size: 0
Max buffer size: 30, current size: 30
class targets: {4, 7}

Max buffer size: 30, current size: 30
class targets: {1, 4, 6, 7}

Max buffer size: 30, current size: 30
class targets: {1, 3, 4, 6, 7, 9}

Max buffer size: 30, current size: 30
class targets: {1, 3, 4, 5, 6, 7, 8, 9}



<ipython-input-12-6a4a72fdc594>:11: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-12-6a4a72fdc594>:11: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-12-6a4a72fdc594>:11: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-12-6a4a72fdc594>:11: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)
<ipython-input-12-6a4a72fdc594>:11: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  storage_p.update(strategy_state)


Max buffer size: 30, current size: 30
class targets: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}



The advantage of using grouping buffers is that you get a balanced rehearsal buffer. You can even access the groups separately with the buffer_groups attribute. Combined with balanced dataloaders, you can ensure that the mini-batches stay balanced during training.

In [ ]:
for k, v in storage_p.buffer_groups.items():
    print(f"(group {k}) -> size {len(v.buffer)}")

(group 4) -> size 3
(group 7) -> size 3
(group 1) -> size 3
(group 6) -> size 3
(group 9) -> size 3
(group 3) -> size 3
(group 8) -> size 3
(group 5) -> size 3
(group 0) -> size 3
(group 2) -> size 3


In [ ]:
datas = [v.buffer for v in storage_p.buffer_groups.values()]
dl = GroupBalancedDataLoader(datas)

for x, y, t in dl:
    print(y.tolist())
    break

[4, 4, 4, 7, 7, 7, 1, 1, 1, 6, 6, 6, 9, 9, 9, 3, 3, 3, 8, 8, 8, 5, 5, 5, 0, 0, 0, 2, 2, 2]


# **Replay Plugins**

Avalanche's strategy plugins can be used to update the rehearsal buffer and set the dataloader. This allows to easily implement replay strategies

In [ ]:
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin

class CustomReplay(SupervisedPlugin):
    def __init__(self, storage_policy):
        super().__init__()
        self.storage_policy = storage_policy

    def before_training_exp(self, strategy,
                            num_workers: int = 0, shuffle: bool = True,
                            **kwargs):
        """ Here we set the dataloader. """
        if len(self.storage_policy.buffer) == 0:
            # first experience. We don't use the buffer, no need to change
            # the dataloader.
            return

        # replay dataloader samples mini-batches from the memory and current
        # data separately and combines them together.
        print("Override the dataloader.")
        strategy.dataloader = ReplayDataLoader(
            strategy.adapted_dataset,
            self.storage_policy.buffer,
            oversample_small_tasks=True,
            num_workers=num_workers,
            batch_size=strategy.train_mb_size,
            shuffle=shuffle)

    def after_training_exp(self, strategy: "BaseStrategy", **kwargs):
        """ We update the buffer after the experience.
            You can use a different callback to update the buffer in a different place
        """
        print("Buffer update.")
        self.storage_policy.update(strategy, **kwargs)

Use the plugin to train our continual model.

In [ ]:
from torch.nn import CrossEntropyLoss
from avalanche.training import Naive
from avalanche.evaluation.metrics import accuracy_metrics
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.models import SimpleMLP
import torch

scenario = SplitMNIST(5)
model = SimpleMLP(num_classes=scenario.n_classes)
storage_p = ParametricBuffer(
    max_size=500,
    groupby='class',
    selection_strategy=RandomExemplarsSelectionStrategy()
)

# choose some metrics and evaluation method
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loggers=[interactive_logger])

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(model, torch.optim.Adam(model.parameters(), lr=0.001),
                    CrossEntropyLoss(),
                    train_mb_size=100, train_epochs=1, eval_mb_size=100,
                    plugins=[CustomReplay(storage_p)],
                    evaluator=eval_plugin
                    )

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience ", experience.current_experience)
    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(scenario.test_stream))

/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


Starting experiment...
Start of experience  0
-- >> Start of training phase << --
100%|██████████| 122/122 [00:04<00:00, 27.69it/s]
Epoch 0 ended.
Buffer update.
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --


<ipython-input-15-04bf1c5bdbc6>:34: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 42.89it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9855
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 35.22it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 41.86it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 35.54it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.0000
-- Starting eval on experience 4 (Task 0) from test stream --
100%|██████████| 22/22 [00:00<

<ipython-input-15-04bf1c5bdbc6>:34: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


...

-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 42.51it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8631
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 35.44it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.8901
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 40.99it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.9782
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 35.73it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/te

<ipython-input-15-04bf1c5bdbc6>:34: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 26.72it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8966
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 38.80it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.8728
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 38.18it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.8928
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 36.82it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/te

<ipython-input-15-04bf1c5bdbc6>:34: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 40.62it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7003
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 35.93it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.8249
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 37.54it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.7026
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 33.43it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.9623
-- Starting eval on experience 4 (Task 0) from test stream -